In [1]:
# {hotel_name} 별로 최대 100개씩 리뷰를 스크랩
# 각 호텔의 디렉터리를 생성하여 scrap_data/{hotel_name}/reviews.txt 에 100개의 리뷰를 저장.
import os
import time
import random
from collections import Counter
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class HotelReviewScraper:
    def __init__(self):
        self.browser = None

    def start_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        service = Service(executable_path=ChromeDriverManager().install())
        self.browser = webdriver.Chrome(service=service, options=options)

    def stop_browser(self):
        if self.browser:
            self.browser.quit()

    def scrap_reviews(self, url, max_reviews=100):
        reviews = []  # 리뷰 수집 리스트

        self.browser.get(url)  # 페이지 로드
        time.sleep(2)  # 초기 로딩을 위한 대기

        scroll_count = 0  # 스크롤 횟수 카운트
        while len(reviews) < max_reviews and scroll_count < 10:
            # 스크롤을 맨 아래로 내림
            self.browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
            time.sleep(random.uniform(1, 2))  # 스크롤 후 대기
            
            # 리뷰 추출
            review_elements = self.browser.find_elements(By.CSS_SELECTOR, "#__next > section > div > div.css-1js0bc8 > div > div:nth-child(3) > div > div:nth-child(5) > div.css-1kpa3g > p")

            # 리뷰 텍스트 수집
            for element in review_elements:
                review = element.text
                if review:  # 빈 리뷰는 제외
                    reviews.append(review)
                    if len(reviews) >= max_reviews:
                        break
            
            # 다음 스크롤을 위해 스크롤 횟수 증가
            scroll_count += 1

        return reviews[:max_reviews]  # 최대 리뷰 개수만 반환

def save_reviews_to_txt(hotel_name, reviews):
    dir_path = f"scrap_data/hotel/{hotel_name}"
    os.makedirs(dir_path, exist_ok=True)  # 호텔 디렉터리 생성
    file_path = os.path.join(dir_path, "reviews.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        for i, review in enumerate(reviews, 1):
            file.write(f"{i}번째 리뷰:\n{review}\n\n")

def main():
    hotel_url_lists = [
        "https://www.yanolja.com/reviews/domestic/3001285",
        "https://www.yanolja.com/reviews/domestic/3008436",
        "https://www.yanolja.com/reviews/domestic/3010282",
        "https://www.yanolja.com/reviews/domestic/3001257",
        "https://www.yanolja.com/reviews/domestic/3001618",
        "https://www.yanolja.com/reviews/domestic/3009057",
        "https://www.yanolja.com/reviews/domestic/3002353",
        "https://www.yanolja.com/reviews/domestic/3008662",
        "https://www.yanolja.com/reviews/domestic/1000111292",
        "https://www.yanolja.com/reviews/domestic/3001255"
    ]
    hotel_name_lists = [
        "제주 봄그리고 가을 호텔&리조트",
        "호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션",
        "호텔위드 제주",
        "베스트웨스턴 제주",
        "호텔 리젠트마린",
        "라마다 제주시티 호텔",
        "더 베스트 제주 성산",
        "호텔샬롬 제주",
        "타마라 호텔 제주",
        "신신호텔 제주 월드컵"
    ]

    scraper = HotelReviewScraper()
    scraper.start_browser()

    try:
        for hotel_name, hotel_url in zip(hotel_name_lists, hotel_url_lists):
            reviews = scraper.scrap_reviews(hotel_url)
            save_reviews_to_txt(hotel_name, reviews)  # 리뷰를 텍스트 파일에 저장
            print(f"Reviews for {hotel_name} saved to scrap_data/hotel/{hotel_name}/reviews.txt")
    finally:
        scraper.stop_browser()

if __name__ == "__main__":
    main()


Reviews for 제주 봄그리고 가을 호텔&리조트 saved to scrap_data/hotel/제주 봄그리고 가을 호텔&리조트/reviews.txt
Reviews for 호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션 saved to scrap_data/hotel/호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션/reviews.txt
Reviews for 호텔위드 제주 saved to scrap_data/hotel/호텔위드 제주/reviews.txt
Reviews for 베스트웨스턴 제주 saved to scrap_data/hotel/베스트웨스턴 제주/reviews.txt
Reviews for 호텔 리젠트마린 saved to scrap_data/hotel/호텔 리젠트마린/reviews.txt
Reviews for 라마다 제주시티 호텔 saved to scrap_data/hotel/라마다 제주시티 호텔/reviews.txt
Reviews for 더 베스트 제주 성산 saved to scrap_data/hotel/더 베스트 제주 성산/reviews.txt
Reviews for 호텔샬롬 제주 saved to scrap_data/hotel/호텔샬롬 제주/reviews.txt
Reviews for 타마라 호텔 제주 saved to scrap_data/hotel/타마라 호텔 제주/reviews.txt
Reviews for 신신호텔 제주 월드컵 saved to scrap_data/hotel/신신호텔 제주 월드컵/reviews.txt


In [2]:
# scrap_data/{hotel_name}/reviews.txt를 읽어서
# {hotel_name}의 리뷰들의 명사와 동사들의 빈도수를 각각 출력
# 그것을 scrap_data/{hotel_name}/nouns_and_verbs.txt에 저장
from collections import Counter
from konlpy.tag import Okt
import os

def save_nouns_and_verbs_to_file(hotel_name, nouns, verbs):
    dir_path = f"scrap_data/hotel/{hotel_name}"
    os.makedirs(dir_path, exist_ok=True)  # 호텔 디렉터리 생성
    file_path = os.path.join(dir_path, "nouns_and_verbs.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(f"명사 빈도수:\n")
        for word, count in nouns:
            file.write(f"{word}: {count}\n")
        file.write(f"\n동사 빈도수:\n")
        for word, count in verbs:
            file.write(f"{word}: {count}\n")
        file.write("\n")

def main():
    hotel_name_lists = [
        "제주 봄그리고 가을 호텔&리조트",
        "호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션",
        "호텔위드 제주",
        "베스트웨스턴 제주",
        "호텔 리젠트마린",
        "라마다 제주시티 호텔",
        "더 베스트 제주 성산",
        "호텔샬롬 제주",
        "타마라 호텔 제주",
        "신신호텔 제주 월드컵"
    ]

    try:
        for hotel_name in hotel_name_lists:
            okt = Okt()
            nouns = Counter()
            verbs = Counter()
            with open(f"scrap_data/hotel/{hotel_name}/reviews.txt", "r", encoding="utf-8") as file:
                reviews = file.readlines()
                for review in reviews:
                    pos = okt.pos(review.strip())
                    nouns.update([word for word, pos_tag in pos if pos_tag.startswith('Noun')])
                    verbs.update([word for word, pos_tag in pos if pos_tag.startswith('Verb')])
                nouns = nouns.most_common(50)
                verbs = verbs.most_common(50)
                print(f"Scraped Reviews for {hotel_name}:")
                save_nouns_and_verbs_to_file(hotel_name, nouns, verbs)
                print(f"  - Nouns and verbs saved for {hotel_name}")
    except FileNotFoundError:
        print("Hotel review files not found.")
        return

if __name__ == "__main__":
    main()


Scraped Reviews for 제주 봄그리고 가을 호텔&리조트:
  - Nouns and verbs saved for 제주 봄그리고 가을 호텔&리조트
Scraped Reviews for 호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션:
  - Nouns and verbs saved for 호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션
Scraped Reviews for 호텔위드 제주:
  - Nouns and verbs saved for 호텔위드 제주
Scraped Reviews for 베스트웨스턴 제주:
  - Nouns and verbs saved for 베스트웨스턴 제주
Scraped Reviews for 호텔 리젠트마린:
  - Nouns and verbs saved for 호텔 리젠트마린
Scraped Reviews for 라마다 제주시티 호텔:
  - Nouns and verbs saved for 라마다 제주시티 호텔
Scraped Reviews for 더 베스트 제주 성산:
  - Nouns and verbs saved for 더 베스트 제주 성산
Scraped Reviews for 호텔샬롬 제주:
  - Nouns and verbs saved for 호텔샬롬 제주
Scraped Reviews for 타마라 호텔 제주:
  - Nouns and verbs saved for 타마라 호텔 제주
Scraped Reviews for 신신호텔 제주 월드컵:
  - Nouns and verbs saved for 신신호텔 제주 월드컵


In [4]:
# scrap_data/{hotel_name}/nouns 와 scrap_data/{hotel_name}/verbs 디렉터리를 생성하고
# scrap_data/{hotel_name}/nouns_and_verbs.txt에서 명사와 동사의 빈도수를 읽어서
# (명사의 빈도수 plot, wordcloud)와 (동사의 빈도수 plot, wordcloud)을 각각
# scrap_data/{hotel_name}/nouns/nouns_word_frequency_plot.png
# scrap_data/{hotel_name}/nouns/nouns_word_wordcloud.png
# scrap_data/{hotel_name}/verbs/verbs_word_frequency_plot.png
# scrap_data/{hotel_name}/verbs/verbs_word_wordcloud.png
# 로 저장.
import os
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud
from konlpy.tag import Okt

def generate_word_frequency_plot_and_word_cloud(hotel_name, nouns, verbs):
    # Create directories if they don't exist
    nouns_dir = f"scrap_data/hotel/{hotel_name}/nouns"
    verbs_dir = f"scrap_data/hotel/{hotel_name}/verbs"
    os.makedirs(nouns_dir, exist_ok=True)
    os.makedirs(verbs_dir, exist_ok=True)

    # 명사 빈도수 플롯
    if nouns:
        words_words, words_counts = zip(*nouns)
        plt.rcParams['font.family'] = 'malgun gothic'
        plt.rcParams['axes.unicode_minus'] = False
        plt.figure(figsize=(12, 6))
        plt.bar(words_words, words_counts, color='skyblue')
        plt.title(f'Word Frequency Plot for {hotel_name} (Nouns)')
        plt.xlabel('Words')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{nouns_dir}/{hotel_name}_nouns_word_frequency_plot.png')
        plt.close()

        # 명사 워드 클라우드 생성
        wc = WordCloud(
            font_path='C:\\Windows\\Fonts\\malgunbd.ttf',
            max_words=100,
            background_color='white'
        )
        word_cloud_img = wc.generate_from_frequencies(dict(nouns))
        word_cloud_img.to_file(f'{nouns_dir}/{hotel_name}_nouns_wordcloud.png')

    # 동사 빈도수 플롯
    if verbs:
        words_words, words_counts = zip(*verbs)
        plt.rcParams['font.family'] = 'malgun gothic'
        plt.rcParams['axes.unicode_minus'] = False
        plt.figure(figsize=(12, 6))
        plt.bar(words_words, words_counts, color='salmon')
        plt.title(f'Word Frequency Plot for {hotel_name} (Verbs)')
        plt.xlabel('Words')
        plt.ylabel('Frequency')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'{verbs_dir}/{hotel_name}_verbs_word_frequency_plot.png')
        plt.close()

        # 동사 워드 클라우드 생성
        wc = WordCloud(
            font_path='C:\\Windows\\Fonts\\malgunbd.ttf',
            max_words=100,
            background_color='white'
        )
        word_cloud_img = wc.generate_from_frequencies(dict(verbs))
        word_cloud_img.to_file(f'{verbs_dir}/{hotel_name}_verbs_wordcloud.png')

def main():
    hotel_name_lists = [
        "제주 봄그리고 가을 호텔&리조트",
        "호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션",
        "호텔위드 제주",
        "베스트웨스턴 제주",
        "호텔 리젠트마린",
        "라마다 제주시티 호텔",
        "더 베스트 제주 성산",
        "호텔샬롬 제주",
        "타마라 호텔 제주",
        "신신호텔 제주 월드컵"
    ]

    for hotel_name in hotel_name_lists:
        nouns = []  # 명사 리스트 초기화
        verbs = []  # 동사 리스트 초기화

        # Read nouns and verbs from file
        with open(f"scrap_data/hotel/{hotel_name}/nouns_and_verbs.txt", "r", encoding="utf-8") as file:
            lines = file.readlines()

        # Extract nouns and verbs data
        found_nouns = False
        found_verbs = False
        for line in lines:
            line = line.strip()
            if line == "명사 빈도수:":
                found_nouns = True
                found_verbs = False
                continue
            elif line == "동사 빈도수:":
                found_nouns = False
                found_verbs = True
                continue
            elif line == "":
                found_nouns = False
                found_verbs = False
                continue
            if found_nouns:
                word, count = line.split(":")
                nouns.append((word.strip(), int(count.strip())))
            elif found_verbs:
                word, count = line.split(":")
                verbs.append((word.strip(), int(count.strip())))

        # Generate word frequency plot and word cloud
        generate_word_frequency_plot_and_word_cloud(hotel_name, nouns, verbs)
        print(f"Word Frequency Plot and Word Cloud saved for {hotel_name}")

if __name__ == "__main__":
    main()


Word Frequency Plot and Word Cloud saved for 제주 봄그리고 가을 호텔&리조트
Word Frequency Plot and Word Cloud saved for 호텔 휘슬락 바이 베스트 웨스턴 시그니처 컬렉션
Word Frequency Plot and Word Cloud saved for 호텔위드 제주
Word Frequency Plot and Word Cloud saved for 베스트웨스턴 제주
Word Frequency Plot and Word Cloud saved for 호텔 리젠트마린
Word Frequency Plot and Word Cloud saved for 라마다 제주시티 호텔
Word Frequency Plot and Word Cloud saved for 더 베스트 제주 성산
Word Frequency Plot and Word Cloud saved for 호텔샬롬 제주
Word Frequency Plot and Word Cloud saved for 타마라 호텔 제주
Word Frequency Plot and Word Cloud saved for 신신호텔 제주 월드컵
